In [15]:
import os
import sys
from datetime import datetime
import shutil
import numpy as np

In [16]:
parent_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(parent_folder)
sys.path.append(parent_folder)

C:\Users\rasmusso\Desktop\preprocessing-mitgcm


In [17]:
from configs.config_object import ConfigObject
from grid_and_bathy import get_dz_grid, get_grid
from surface_forcings import extract_and_save_surface_forcings, download_weather_reanalysis, download_weather_forecast
from initial_conditions import download_profile_idronaut_datalakes, create_initial_temperature_from_measure_profile, parse_alplakes_1d_from_directory
from run_preprocessing import write_data_config_files, copy_template, remove_all_files_and_folders, write_size_config_files
from utils import modify_arguments, convert_binary_files

## Load config

In [18]:
config = ConfigObject('configs/config.json')
output_folder = './99-output/'
template_folder = os.path.join('./00-template_mitgcm/', config.template_folder)
print(template_folder)

./00-template_mitgcm/zurich_base


In [19]:
parsed_start_date = datetime.strptime(config.start_date, '%Y%m%d')
parsed_end_date = datetime.strptime(config.end_date, '%Y%m%d')
parsed_ref_date = datetime.strptime(config.reference_date, '%Y%m%d')

sim_duration_in_second = (parsed_end_date - parsed_start_date).total_seconds()
start_time_in_second_from_ref_date = (parsed_start_date - parsed_ref_date).total_seconds()
end_time_in_second_from_ref_date = (parsed_end_date - parsed_ref_date).total_seconds()

## Initialize output folder

In [20]:
remove_all_files_and_folders(output_folder)
#copy_template('./00-template_mitgcm/geneva_base', output_folder)
copy_template(template_folder, output_folder)
config.write_metadata_to_file(os.path.join(output_folder, 'metadata_simulation.txt'))

## Get MITgcm grid

#In case the grid has not been created yet
build_and_save_mitgcm_grid(config.paths.grid_folder_path, 
                           config.Nx, 
                           config.Ny, 
                           config.grid_resolution, 
                           config.x0_ch1903, 
                           config.y0_ch1903, 
                           config.x1_ch1903, 
                           config.y1_ch1903)

In [21]:
mitgcm_grid = get_grid(config.paths.grid_folder_path)

## Build binary files

In [22]:
binary_data_folder = os.path.join(output_folder, 'binary_data')
if not os.path.exists(binary_data_folder):
    os.makedirs(binary_data_folder)
    print(f"Directory '{binary_data_folder}' created.")

Directory './99-output/binary_data' created.


**Get bathymetry**

In [23]:
#To create bathymetry, see notebook 'create_bathy'
shutil.copy(config.paths.bathy_path, os.path.join(binary_data_folder, 'bathy.bin'))

'./99-output/binary_data\\bathy.bin'

**Get surface forcings**
run to dowload external forcing.
Once downloaded, no need to run that cell again

In [24]:
buffer = config.weather_download_buffer
if config.weather_model_type == 'reanalysis':
    download_weather_reanalysis(config.weather_api_base_url, 
                                config.start_date, config.end_date, 
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_results_from_api_folder)
    print('Finished downloading weather reanalysis')
elif config.weather_model_type == 'forecast':
    download_weather_forecast(config.weather_api_base_url, 
                                config.start_date,  
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_results_from_api_folder)
    print('Finished downloading weather forecasts')

Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_T_2M.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_U.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_GLOB.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_V.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_RELHUM_2M.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_PMSL.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_CLCT.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240301_20240301_PS.json
Data already exists: C:\Users\rasmusso\Documents\ZH\data\meteo\continuous reanalysis\20240302_20240

In [25]:
time_format = 'UTC'
extract_and_save_surface_forcings(binary_data_folder, 
                                  config.start_date, 
                                  config.end_date, 
                                  config.paths.raw_results_from_api_folder, 
                                  mitgcm_grid,
                                  1, # Number of cores used in interpolation
                                  config.weather_model_type)

Interpolating U to grid...
Saving u10...
Interpolating V to grid...
Saving v10...
Interpolating GLOB to grid...
Saving swdown...
Interpolating T_2M to grid...
Saving atemp...
Interpolating PS to grid...
Saving apressure...
Computing specific humidity (aqh)...
Interpolating RELHUM_2M to grid...
Saving relhum...
Saving aqh...
Computing longwave radiation (lwdown)...
Interpolating CLCT to grid...
Saving clct...
Saving lwdown...
Done computing binary data.


**Convert to little endian if necessary** (for cscs computation)

In [26]:
# TO DO : implement check to see whether the datatypes are correct
if config.endian_type == 'little_endian':
    convert_binary_files(os.path.join(output_folder, 'binary_data'), '>f8', '<f8')

### Get initial conditions

file_date, measured_temperature = download_profile_idronaut_datalakes(parsed_start_date)
print('Initial conditions from Idronaut file from ', file_date)
dz_grid = get_dz_grid(config.paths.dz_grid_csv_path)
shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, measured_temperature)

In [27]:
# From Simstrat results (Aplakes)
simstrat_temperature = parse_alplakes_1d_from_directory(r'./initial_conditions/simstrat_data')
profile_simstrat = simstrat_temperature.sel(time = parsed_start_date, method='nearest')
print('Initial conditions from Simstrat ', profile_simstrat.time.values)
dz_grid = get_dz_grid(config.paths.dz_grid_csv_path)
dz_grid = dz_grid[~np.isnan(dz_grid)]
shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, profile_simstrat)

Initial conditions from Simstrat  2024-03-01T01:00:02.880000000


### Get pickup files

In [30]:
if config.with_pickup:
    dt_sim_in_second = config.time_step
    pickup_number = f"pickup.{str(int(start_time_in_second_from_ref_date / dt_sim_in_second)).zfill(10)}"
else:
    pickup_number = ""
print(pickup_number)

### Adapt config files

In [31]:
modify_arguments('!reference_date!', [config.reference_date], os.path.join(output_folder, 'run_config/data.cal'))

In [32]:
modify_arguments('!start_date!', [config.start_date], os.path.join(output_folder, 'run_config/data.exf'))

In [44]:
#TO DO: write output type (binary or netcdf -->   diag_mnc = .FALSE./.TRUE. in data.diagnostics)
len(shaped_temp_initial)
str_values=''
for row in shaped_temp_initial:
    str_values += str(str(row) + ',')
    str_values += '\n'
print(str_values)

6.323,
6.32706,
6.329184000000001,
6.331188,
6.3323160000000005,
6.333,
6.333,
6.331626,
6.32867,
6.319472,
6.3031299999999995,
6.281428,
6.2555439999999995,
6.234151,
6.218086,
6.207749,
6.2004410000000005,
6.194484,
6.1884429999999995,
6.18168,
6.174344,
6.167186,
6.160368,
6.1541,
6.148925,
6.143595,
6.13868,
6.134148,
6.1294759999999995,
6.12466,
6.118392,
6.102715,
6.079386,
6.058052,
6.039121,
6.021603,
6.005235,
5.989659,
5.9747,
5.9607600000000005,
5.945679999999999,
5.931824,
5.9175439999999995,
5.903824,
5.890236000000001,
5.877755,
5.86768,
5.858977,
5.853036,
5.848622,
5.846036,
5.844691,
5.843274,
5.843,
5.8422149999999995,
5.842,
5.842,
5.842,
5.842,
5.842,
5.842,
5.841,
5.841,
5.841,
5.841,
5.841,
5.841,
5.841,
5.841,
5.841,
5.841,
5.841,
5.842,
5.842,



In [42]:
write_data_config_files(os.path.join(output_folder, 'run_config/data'), 
                        shaped_temp_initial, 
                        start_time_in_second_from_ref_date, 
                        end_time_in_second_from_ref_date,
                        pickup_number,
                        dz_grid, 
                        config.grid_resolution,
                        config.time_step)

TypeError: 'numpy.float64' object is not iterable

In [40]:
use_exch2=False
# Package to ignore cores that are attributed to land. See notebook grid_and_bathy/check_core_allocation to create files land_cores_Px..etc.txt.
# Doc here: https://wiki.math.uwaterloo.ca/fluidswiki/index.php?title=MITgcm_tips#Setting_the_equation_of_state (look for exch2)
# And there: https://mitgcm.readthedocs.io/en/latest/phys_pkgs/exch2.html (not so helpful... don't we love the official documentation of MITgcm?)
if use_exch2:    
    blank_list_path=os.path.join(config.paths.grid_folder_path, f"land_cores_Px{config.Px}_Py{config.Py}.txt")
    with open(blank_list_path, "r") as file:
        nb_blank_cores, blank_list = file.read().splitlines()
        
    Px = (config.Px * config.Py) - int(nb_blank_cores)
    Py = 1
    
    modify_arguments('!Nx!', [config.Nx], os.path.join(output_folder, 'run_config/data.exch2'))
    modify_arguments('!Ny!', [config.Ny], os.path.join(output_folder, 'run_config/data.exch2'))
    modify_arguments('!blank_list!', [blank_list], os.path.join(output_folder, 'run_config/data.exch2'))
else:
    Px = config.Px
    Py = config.Py

In [41]:
write_size_config_files(os.path.join(output_folder, 'code/SIZE.h'), 
                        Px, 
                        Py, 
                        config.Nx, 
                        config.Ny, 
                        len(dz_grid.flatten()))

## Next steps

- Copy-paste the folders in "99-output" to the cscs server 
- Change #SBATCH --time=00:05:00 in run/submit-daint-short.sh
- Open CSCS putty console:

export MITGCM_ROOTDIR=/scratch/snx3000/aleroqua/MITgcm-checkpoint67z/

cd /scratch/snx3000/aleroqua/xx/build
$MITGCM_ROOTDIR/tools/genmake2 -mods ../code -mpi -enable=mnc -of ${MITGCM_ROOTDIR}/tools/build_options/linux_ia64_cray_archer
make depend
make -j 8

cd ..
rm -r run
mkdir run
cd run
ln -s ../pickup/* .
ln -s ../run_config/* .
cp ../build/mitgcmuv .

sbatch submit-daint-short.sh

### TO DO

In [ ]:
# add selection of pickup file
# add selection of pickup frequency, diagnostic frequency
# add selection of output type (mnc = true or false)
# add automatic change of computing time? (submit-daint)